<a href="https://colab.research.google.com/github/rpizarrog/probabilidad-y-estad-stica/blob/master/2024/Caso_36_Regresion_lineal_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo

Evaluar y realizar predicciones con un modelo de regresión lineal simple.





# Descripción

* Cargar librerias
* Cargar funciones
* Cargar datos
* Construir modelo de regresión lineal simple
* Evaluar modelo con *root mean squared error RMSE*.
* Hacer predicciones
* Interpretar el caso

# Fundamento teórico

La regresión lineal simple implica aplicar una ecuación matemática de mínimos cuadrados que permite pronosticar o predecir el valor de una variable con base en el valor de otra; este procedimiento se llama análisis de regresión.

El análisis de regresión es un método para examinar una relación lineal entre dos variables; se utiliza el concepto de correlación r
, sin embargo, la regresión proporciona mucho más información, además de permitir estimaciones o predicciones de la relación lineal con la ecuación de mínimos cuadrados (Lind, Marchal, and Wathen 2015).

De auerdo a chat gpt: La regresión lineal simple es un método estadístico utilizado para modelar la relación entre dos variables: una independiente (predictora) y una dependiente (respuesta). La relación se representa mediante una línea recta, cuya ecuación es $𝑦=𝑚𝑥+a$, donde 𝑦 es la variable dependiente, 𝑥 es la variable independiente, 𝑚 es la pendiente de la línea, y
$a$ es la intersección con el eje 𝑦.

Este modelo ayuda a predecir valores de
𝑦 basados en valores de 𝑥 y a entender la relación entre las dos variables.

Fórmula de mínimos cuadrados:

$$
Y = a + bx
$$

* $Y$ Es el valor de una predicccón
* $a$ es el coeficiente de la abcisa
* $b$ es el valor de la pendiente de la recta de regresión
* $x_i$ son los valores de la variable independiente $x$ que permiten hacer prediciones.

# Interpretación



# Bibliografía

* Amat Rodrigo, Joaquín. 2016. “Correlación Lineal y Regresión Lineal Simple.” 2016. https://www.cienciadedatos.net/documentos/24_correlacion_y_regresion_lineal.
* OpenAI. (2024, Mayo 28). Respuesta generada por ChatGPT [Respuesta a consulta]. Obtenido de https://www.openai.com/chatgpt
* Hernández Sampieri, Roberto, Carlos Fernández Collado, and María del Pilar Baptista Lucio. 2014. Metodología de La Investigación. Sexta.
* Lind, Douglas, William Marchal, and Samuel Wathen. 2015. Estadística Aplicada a Los Negocios y La Economía. Decimo Sexta. México, D.F.: McGraw-Hill.
* Walpole, Ronald E., Raymond H. Myers, and Sharon L. Myers. 2012. Probabilidad y Estadística Para Ingeniería y Ciencias. Novena Edición. México: Pearson.
* Walpole, Ronald E., Raymond H. Myers, Sharon L. Myers, and Keying Ye. 2007. Probabilidad y Estadística Para Ingeniería y Ciencias. Octava Edición. México: Pearson Education.